In [1]:
import fluids as f
from ipywidgets import interact
from math import pi

DEFAULT_MATERIAL='galvanised steel'
atm = f.atmosphere.ATMOSPHERE_1976(Z=50)

# Ductwork Calc (using fluids library)

In [2]:
from pydantic import BaseModel, Field, computed_field, field_validator, ValidationInfo, model_validator
import ipywidgets as w
from ipyautoui import AutoUi
from ipyautoui.autoobject import AutoObject
import traitlets as tr
import fluids as f
from fluids.friction import roughness_clean_names 
from math import pi
from IPython.display import clear_output

DEFAULT_MATERIAL = 'Riveted steel'
DEFAULT_ATM = f.atmosphere.ATMOSPHERE_1976(Z=50)

def calc_pressure_drop(q, d_mm, material_roughness):
    # https://fluids.readthedocs.io/fluids.friction.html#pressure-drop-calculation
    return f.friction.one_phase_dP(m=q/DEFAULT_ATM.rho,
                               rho=DEFAULT_ATM.rho,
                               mu=DEFAULT_ATM.mu,
                               D=d_mm/1000,
                               roughness=material_roughness,
                               L=1)

def calc_velocity(q, d_mm):
    d = d_mm / 1000
    csa = .25 * pi * d**2
    return q / csa

def calc_dia(q, material_roughness, velocity_max=6, pressure_drop_max=0.4):
    dia_mm = 0
    dp = pressure_drop_max + 1
    v = velocity_max + 1
    while dp > pressure_drop_max or v > velocity_max: 
        dia_mm+=50
        dp = calc_pressure_drop(q, dia_mm, material_roughness)
        v = calc_velocity(q, dia_mm)
    return q, dia_mm, calc_velocity(q, dia_mm), calc_pressure_drop(q, dia_mm, material_roughness)


TARGET_VELOCITIES = {
    "0<{}<=3": "terminal branches", 
    "3<{}<=6": "risers",
    "6<{}<=10": "avoid - too fast"
}

class VelocityUi(w.VBox):
    _value = tr.Float(default_vavlue=0.1)
    
    def __init__(self,value=0.1,  **kwargs):
        super().__init__(**kwargs | {"layout":{"border": "solid 1px yellow"}}) #  | {"layout":{"width": "500px"}}
        self.velocity = w.FloatSlider(min=0, max=10)
        self.label = w.HTML()
        self.children = [self.velocity, self.label]
        self._init_controls()
        self._update("")
        self.value = value

    @property
    def value(self):
        return self._value

    @value.setter
    def value(self, value):
        self.velocity.value = value
        
    def _init_controls(self):
        self.velocity.observe(self._update, "value")

    def _update(self, on_change):
        self._value = self.velocity.value
        for k, v in TARGET_VELOCITIES.items():
            if eval(k.format(self.value)):
                self.label.value = "^ <i>"+ v + "</i>"
                break
        

class PressureDropIn(BaseModel):
    material: str = Field('Riveted steel', examples=roughness_clean_names)
    material_roughness: float = Field(f.material_roughness(DEFAULT_MATERIAL), json_schema_extra=dict(disabled=True))
    velocity_max: float = Field(1, title= "Velocity max (m/s)", json_schema_extra=dict(autoui="__main__.VelocityUi"))
    pressure_drop_max: float = Field(0.4, ge=0.4, le=1.5, title="Pressure drop max (kPa)")
    flowrate_design: float = Field(1, ge=0, le=10, title="flowrate (m3/s)", json_schema_extra=dict(step=0.01)) 



    @field_validator('material_roughness')
    def calc_material_roughness(cls, v, info: ValidationInfo):
        material = info.data.get("material")
        return f.material_roughness(material)

class PressureDropOut(BaseModel):
    velocity_design: float = Field(1, minimum=0, maximum=10) 
    pressure_drop_design: float 
    dia_mm: float = Field(0)


class DuctSizeCalc(w.HBox):
    def __init__(self):
        self.ui_in = w.VBox([w.HTML("<b>Inputs</b>"), AutoObject.from_pydantic_model(PressureDropIn, show_savebuttonbar=False, show_validation=False, show_status=False)])
        self.ui_out = w.VBox([w.HTML("<b>Outputs</b>"), AutoObject.from_pydantic_model(PressureDropOut, show_savebuttonbar=False, disabled=True)])
        self.out = w.Output()
        super().__init__([self.ui_in, self.ui_out])
        self._init_controls()
        self._calc("")

    def _init_controls(self):
        self.ui_in.observe(self._calc, "_value")

    def _calc(self, on_change):
        with self.out:
            clear_output()
        try:
            flowrate_design, material_roughness, velocity_max, pressure_drop_max = (
                self.ui_in.value.get("flowrate_design"), 
                self.ui_in.value.get("material_roughness"), 
                self.ui_in.value.get("velocity_max"), 
                self.ui_in.value.get("pressure_drop_max")
            )
            di = {}
            q, di['dia_mm'], di['velocity_design'], di['pressure_drop_design'] = calc_dia(flowrate_design, material_roughness, velocity_max=velocity_max, pressure_drop_max=pressure_drop_max)
            self.ui_out.value = PressureDropOut(**di).model_dump()
        except:
            with self.out:
                print("error")

calc = DuctSizeCalc()

calc
# titles = ["Ductwork Sizing", "Pipework Sizing", "Pump Pressure Drop", "Expansion Vessel Sizing", "Heat Pump and Buffer Vessel Sizing", "Psychrometrics"]
# widgets = [w.HTML(t) for t in titles]
# widgets[0] = calc
# acc = w.Accordion(
#     widgets, titles=titles
# )
# acc

/tmp/ipykernel_141113/1523570680.py:91: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'minimum', 'maximum'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  velocity_design: float = Field(1, minimum=0, maximum=10)


DuctSizeCalc(children=(VBox(children=(HTML(value='<b>Inputs</b>'), AutoObject(children=(VBox(children=(AutoBox…

In [3]:
# import numpy as np
# import math

# def roundup(x, nearest=100):
#     return int(math.ceil(x / nearest)) * nearest

# def calc_duct_height(area, width, nearest=100):
#     return roundup((area / (width/1000))*1000)

# class RectangularDuct(BaseModel):
#     area: float
#     width: float
#     height: float = Field(0, validate_default=True)
 

#     @field_validator('height')
#     def calc_height(cls, v, info: ValidationInfo):
#         area, width = info.data.get("area"), info.data.get("width")
#         return calc_duct_height(area, width)

# dia_mm = calc.ui_out.value.get("dia_mm")
# area = pi * (dia_mm/2000)**2
# width = 200
# RectangularDuct(area=area, width=width)


